In [12]:
%pylab inline
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from IPython.core.magic import register_cell_magic
import pandas as pd
import numpy as np
import xgboost

Populating the interactive namespace from numpy and matplotlib


In [15]:
bechdelDataset = pd.read_csv('mergeWithRevenue.csv', encoding='utf-8') 
bechdelDataset.head(10)

,Unnamed: 0,id,imdb_id,Rating,Title,Year,Unnamed: 0_y,color,director_name,num_critic_for_reviews,...,producer,writer,box_office,parental_rating,production_company,adjusted_box_office,Multigenre,director_name%,writer%,producer%
0,0,5433,3.0,0,Pauvre Pierrot,1892,NaN,NaN,í_óëóí_ëíí_óëóí_ëí_óëóí_ë...,NaN,...,Julien Pappí_óëóí_ëíí_óëóí_ëí_...,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.0,0.000000
1,1,5444,14.0,0,Tables Turned on the Gardener,1895,NaN,NaN,Louis Lumií_óëóí_ëíí_óëóí_ëí_ó...,NaN,...,Louis Lumií_óëóí_ëíí_óëóí_ëí_ó...,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.0,0.000000
2,2,6200,132134.0,0,"Execution of Mary, Queen of Scots, The",1895,NaN,NaN,Alfred Clark,NaN,...,Thomas A. Edison,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.0,0.000000
3,3,6199,12.0,0,"Arrival of a Train, The",1896,NaN,NaN,Auguste Lumií_óëóí_ëíí_óëóí_ëí_...,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.0,0.000000
4,4,5406,131.0,0,Une nuit terrible,1896,NaN,NaN,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,NaN,...,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,NaN,NaN,NaN,NaN,0.0,Y,0.032258,0.0,0.058824
5,5,4982,91.0,0,"House of the Devil, The",1896,NaN,NaN,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,NaN,...,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,NaN,NaN,NaN,0.0,Y,0.032258,0.0,0.058824
6,6,5445,223341.0,0,La fee aux choux,1896,NaN,NaN,Alice Guy,NaN,...,Lí_óëóí_ëíí_óëóí_ëí_óëóí_...,Alice Guy,NaN,NaN,NaN,0.0,Y,0.000000,0.0,0.000000
7,7,5411,224240.0,0,"Temptation of St. Anthony, The",1898,NaN,NaN,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,Y,0.032258,0.0,0.000000
8,8,5410,211.0,0,"Astronomer&#39;s Dream; or, The Man in the Moo...",1898,NaN,NaN,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,NaN,...,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,NaN,NaN,Kadokawa Pictures,0.0,Y,0.032258,0.0,0.058824
9,9,4978,135696.0,0,Four Heads Are Better Than One,1898,NaN,NaN,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,NaN,...,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,NaN,NaN,NaN,NaN,0.0,Y,0.032258,0.0,0.058824


In [10]:
#use this to construct pass percentages for each director/writer/producer 
#over their work in the dataset
from collections import defaultdict

def returnValue(x, counter):
    if (pd.notnull(x)):
       return counter[x]
    else:
       return 0.0

def constructRecord(dataset, role):
    columnName = role
    passValues = dataset[dataset.binary == 'PASS'][columnName].value_counts().to_dict()
    totalValues = dataset[columnName].value_counts().to_dict()
    passDict = defaultdict(float)
    for value in passValues:
        passDict[value] = passValues[value] / totalValues[value]
           
    dataset[columnName+'%'] = dataset[columnName].apply(lambda x: returnValue(x, passDict))  
    return dataset


In [12]:
bechdelDataset = constructRecord(bechdelDataset, 'director_name')
bechdelDataset = constructRecord(bechdelDataset, 'writer')
bechdelDataset = constructRecord(bechdelDataset, 'producer')

In [13]:
bechdelDataset.head(40)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,index,id,...,producer,writer,box_office,parental_rating,production_company,adjusted_box_office,Multigenre,director_name%,writer%,producer%
0,0,0,0,0,0,0,0,0,0,5433,...,Julien Pappííëíííí«ííí...,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.000000,0.000000
1,1,1,1,1,1,1,1,1,1,5444,...,Louis Lumiííëíííí«ííí...,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.000000,0.000000
2,2,2,2,2,2,2,2,2,2,6200,...,Thomas A. Edison,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.000000,0.000000
3,3,3,3,3,3,3,3,3,3,6199,...,NaN,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.000000,0.000000
4,4,4,4,4,4,4,4,4,4,5406,...,Georges Mííëíííí«íííí...,NaN,NaN,NaN,NaN,0.0,Y,0.032258,0.000000,0.058824
5,5,5,5,5,5,5,5,5,5,4982,...,Georges Mííëíííí«íííí...,Georges Mííëíííí«íííí...,NaN,NaN,NaN,0.0,Y,0.032258,0.000000,0.058824
6,6,6,6,6,6,6,6,6,6,5445,...,Lííëíííí«ííííëíí...,Alice Guy,NaN,NaN,NaN,0.0,Y,0.000000,0.000000,0.000000
7,7,7,7,7,7,7,7,7,7,5411,...,NaN,NaN,NaN,NaN,NaN,0.0,Y,0.032258,0.000000,0.000000
8,8,8,8,8,8,8,8,8,8,5410,...,Georges Mííëíííí«íííí...,Georges Mííëíííí«íííí...,NaN,NaN,Kadokawa Pictures,0.0,Y,0.032258,0.000000,0.058824
9,9,9,9,9,9,9,9,9,9,4978,...,Georges Mííëíííí«íííí...,NaN,NaN,NaN,NaN,0.0,Y,0.032258,0.000000,0.058824


In [14]:
bechdelDataset.to_csv('mergeWithRevenue.csv', encoding='utf-8')

In [23]:
names = bechdelDataset['producer'].unique()
producerFirstNames = []
for name in names:
    if (pd.notnull(name)):
        first = name.split(' ')
        producerFirstNames.append(first[0])
        
print(len(set(producerFirstNames)))

1792


In [25]:
names = bechdelDataset['writer'].unique()
writerFirstNames = []
for name in names:
    if (pd.notnull(name)):
        first = name.split(' ')
        writerFirstNames.append(first[0])
        
print(len(set(writerFirstNames)))

1935


In [26]:
names = bechdelDataset['director_name'].unique()
producerFirstNames = []
for name in names:
    if (pd.notnull(name)):
        first = name.split(' ')
        producerFirstNames.append(first[0])
        
print(len(set(producerFirstNames)))

1599


In [18]:
bechdelDataset['director_gender'] = pd.Series(['Y' for i in range(len(bechdelDataset))])
bechdelDataset['producer_gender'] = pd.Series(['Y' for i in range(len(bechdelDataset))])
bechdelDataset['writer_gender'] = pd.Series(['Y' for i in range(len(bechdelDataset))])

In [19]:
bechdelDataset.head(10)

,Unnamed: 0,id,imdb_id,Rating,Title,Year,Unnamed: 0_y,color,director_name,num_critic_for_reviews,...,parental_rating,production_company,adjusted_box_office,Multigenre,director_name%,writer%,producer%,director_gender,producer_gender,writer_gender
0,0,5433,3.0,0,Pauvre Pierrot,1892,NaN,NaN,í_óëóí_ëíí_óëóí_ëí_óëóí_ë...,NaN,...,NaN,NaN,0.0,Y,0.000000,0.0,0.000000,Y,Y,Y
1,1,5444,14.0,0,Tables Turned on the Gardener,1895,NaN,NaN,Louis Lumií_óëóí_ëíí_óëóí_ëí_ó...,NaN,...,NaN,NaN,0.0,Y,0.000000,0.0,0.000000,Y,Y,Y
2,2,6200,132134.0,0,"Execution of Mary, Queen of Scots, The",1895,NaN,NaN,Alfred Clark,NaN,...,NaN,NaN,0.0,Y,0.000000,0.0,0.000000,Y,Y,Y
3,3,6199,12.0,0,"Arrival of a Train, The",1896,NaN,NaN,Auguste Lumií_óëóí_ëíí_óëóí_ëí_...,NaN,...,NaN,NaN,0.0,Y,0.000000,0.0,0.000000,Y,Y,Y
4,4,5406,131.0,0,Une nuit terrible,1896,NaN,NaN,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,NaN,...,NaN,NaN,0.0,Y,0.032258,0.0,0.058824,Y,Y,Y
5,5,4982,91.0,0,"House of the Devil, The",1896,NaN,NaN,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,NaN,...,NaN,NaN,0.0,Y,0.032258,0.0,0.058824,Y,Y,Y
6,6,5445,223341.0,0,La fee aux choux,1896,NaN,NaN,Alice Guy,NaN,...,NaN,NaN,0.0,Y,0.000000,0.0,0.000000,Y,Y,Y
7,7,5411,224240.0,0,"Temptation of St. Anthony, The",1898,NaN,NaN,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,NaN,...,NaN,NaN,0.0,Y,0.032258,0.0,0.000000,Y,Y,Y
8,8,5410,211.0,0,"Astronomer&#39;s Dream; or, The Man in the Moo...",1898,NaN,NaN,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,NaN,...,NaN,Kadokawa Pictures,0.0,Y,0.032258,0.0,0.058824,Y,Y,Y
9,9,4978,135696.0,0,Four Heads Are Better Than One,1898,NaN,NaN,Georges Mí_óëóí_ëíí_óëóí_ëí_ó...,NaN,...,NaN,NaN,0.0,Y,0.032258,0.0,0.058824,Y,Y,Y
